<a href="https://colab.research.google.com/github/sophia-moore/232-Final-Project/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Data Loading and Exploration

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from typing import Sequence, Dict, Tuple, Optional
from collections import Counter


In [ ]:
# Read the pickle file
file_path = r"C:\Users\gpapa\OneDrive\My Life\Education\2025 YALE MMS\Term_4S\T4_E_Adv_Lin_Algebra_p2\final_project\DJN_2017-01.pkl"
with open(file_path, 'rb') as f:
    data = pickle.load(f)

C:\Users\gpapa\AppData\Local\Temp\ipykernel_25504\3671339968.py:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)


In [ ]:
# Display basic information about the dataset
print(type(data))
if isinstance(data, pd.DataFrame):
    print(data.head())
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
       Date   Company                                              Title  \
0  20170101  [ATC.AE]  [press, release, murphy, owner, kxly, abc, cha...   
1  20170101    [ARKR]  [ark, restaurant, apos, ceo, weinstein, result...   
2  20170101       [F]  [like, futurist, be, prepare, totally, unexpec...   
3  20170101    [NCOM]  [press, release, national, commerce, corporati...   
4  20170101     [EDE]  [press, release, district, electric, company, ...   

                                             Article  Compound_Return  \
0  [remove, it, programming, wire, despite, willi...              NaN   
1                             [from, seek, earnings]         0.070817   
2  [by, in, resident, futurist, lead, team, imagi...         0.076860   
3  [part, family, globe, ncc, parent, company, he...        -0.020161   
4  [wire, closing, today, merger, company, subsid...              NaN   

   Day1_Return  
0          NaN  
1     0.008468  
2     0.046068 

In [ ]:
# Calculate average length of title lists
avg_title_length_list = data['Title'].apply(len).mean()
print(f"Average length of titles: {avg_title_length_list:.2f} words")


Average length of titles: 9.08 words


In [ ]:
# Calculate average length of ARTICLE lists
avg_article_length_list = data['Article'].apply(len).mean()
print(f"Average length of Articles: {avg_article_length_list:.2f} words")

Average length of Articles: 210.09 words


In [ ]:
unique_dates = data['Date'].value_counts()
unique_dates.sort_index(ascending=True, inplace=True)


In [ ]:
unique_dates
# our dates span from 2017-01-01 to 2017-01-31 => ONE MONTH

Date
20170101       6
20170102      47
20170103    1340
20170104    2023
20170105    1795
20170106     988
20170107      29
20170108      42
20170109    1136
20170110    1260
20170111    1102
20170112    1185
20170113     790
20170114      20
20170115      35
20170116     348
20170117    1376
20170118    1402
20170119    1510
20170120     832
20170121      14
20170122      43
20170123    1124
20170124    1424
20170125    1562
20170126    1744
20170127    1058
20170128      21
20170129      40
20170130    1367
20170131    1750
Name: count, dtype: int64

In [ ]:

# df = pd.read_csv("path/to/your/dataset.csv")   # <- uncomment & adapt if you haven’t loaded it yet
assert {'Article', 'Day1_Return'}.issubset(data.columns), "Missing one of the required columns."

# 3. Keep only observations with a target value
clean_df = data.dropna(subset=['Day1_Return']).reset_index(drop=True)

base = clean_df.drop_duplicates(subset='Article')

# 4-A.  IID (shuffled) 80 / 20 split  -------------------------
#      (Most common choice for cross-sectional text problems)
X_train, X_test, y_train, y_test = train_test_split(
    base['Article'],             # features (raw tokens for now)
    base['Day1_Return'],         # target
    test_size=0.25,                  # 25 % hold-out
    random_state=42,                 # reproducible shuffling
    shuffle=True                     # default; keeps rows IID
)

# ------------------------------------------------------------
# 4-B.  Time-ordered split  (uncomment if you prefer it)
#      Keeps entire “future” days completely out-of-sample.
# ------------------------------------------------------------
# clean_df['Date'] = pd.to_datetime(clean_df['Date'].astype(str))  # yyyymmdd → Timestamp
# clean_df = clean_df.sort_values('Date').reset_index(drop=True)
# split_idx   = int(0.80 * len(clean_df))
# X_train     = clean_df.loc[:split_idx-1, 'Article']
# y_train     = clean_df.loc[:split_idx-1, 'Day1_Return']
# X_test      = clean_df.loc[split_idx:,   'Article']
# y_test      = clean_df.loc[split_idx:,   'Day1_Return']

# 5.  Quick sanity check
print(f"Rows kept (non-duplicate articles): {len(base):,}")
print(f"Train size               : {len(X_train):,}")
print(f"Test  size               : {len(X_test):,}")

Rows kept (non-duplicate articles): 11,171
Train size               : 8,378
Test  size               : 2,793


**Rows kept with deduplicates (non-null target): 21,604  
Train size               : 16,203  
Test  size               : 5,401**

In [ ]:
dirty_articles_counts = data['Article'].value_counts()
dirty_articles_counts

Article
[insider, surrender, share, back, company, pay, tax, cover, cost, option, exercise, file, insider, transaction, company, apos, share, open, market, purchase, sale, must, report, within, two, business, day, transaction, end]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  1799
[form, file, reflect, intention, holder, restrict, stock, share, after, mail, filer, permi

In [ ]:
#check how many duplicate values in Article column are there
duplicates = clean_df['Article'].duplicated().sum()
print(f"Number of duplicate articles: {duplicates:,}")

Number of duplicate articles: 10,433


In [ ]:
# Count unique articles that have duplicates
article_counts = clean_df['Article'].value_counts()
print(f"Number of unique articles: {len(article_counts):,}")
print('\n')
different_duplicates = len(article_counts[article_counts > 1])
print(f"Number of unique articles that have duplicates: {different_duplicates}")

Number of unique articles: 11,171


Number of unique articles that have duplicates: 421


In [ ]:
clean_df['Article'].value_counts()

Article
[insider, surrender, share, back, company, pay, tax, cover, cost, option, exercise, file, insider, transaction, company, apos, share, open, market, purchase, sale, must, report, within, two, business, day, transaction, end]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              1785
[form, file, reflect, intention, holder, restrict, stock, shar

In [ ]:
# Get the most common article and its details
most_common_article = clean_df['Article'].value_counts().index[0]

# Convert the lists to tuples for comparison (lists aren't hashable)
matching_mask = clean_df['Article'].apply(lambda x: tuple(x) == tuple(most_common_article))
matching_rows = clean_df[matching_mask][['Date', 'Company', 'Day1_Return']]

print("\nDetails for the most frequent article:")
print(matching_rows)
print(f"\nThis article appears {len(matching_rows)} times in the dataset")

# Optional: Print the actual article content (as a list of tokens)
print("\nArticle content (tokens):")
print(most_common_article)


Details for the most frequent article:
           Date Company  Day1_Return
90     20170103  [TMUS]    -0.020571
334    20170103   [QTS]     0.002887
343    20170103   [QTS]     0.002887
352    20170103   [QTS]     0.002887
376    20170103   [QTS]     0.002887
...         ...     ...          ...
21548  20170131   [AMN]     0.024793
21561  20170131  [VSAR]     0.040590
21563  20170131  [VSAR]     0.040590
21566  20170131  [VSAR]     0.040590
21567  20170131  [AMGN]     0.049818

[1785 rows x 3 columns]

This article appears 1785 times in the dataset

Article content (tokens):
['insider', 'surrender', 'share', 'back', 'company', 'pay', 'tax', 'cover', 'cost', 'option', 'exercise', 'file', 'insider', 'transaction', 'company', 'apos', 'share', 'open', 'market', 'purchase', 'sale', 'must', 'report', 'within', 'two', 'business', 'day', 'transaction', 'end']


In [ ]:
# Count unique articles that have duplicates
base_counts = base['Article'].value_counts()
print(f"Number of unique articles: {len(base_counts):,}")
print('\n')
different_base_duplicates = len(base_counts[base_counts > 1])
print(f"Number of unique articles that have duplicates: {different_base_duplicates}")

Number of unique articles: 11,171


Number of unique articles that have duplicates: 0


In [ ]:
X_train.head(10)

2083     [operate, direct, insider, surrender, share, b...
10443    [more, follow, press, release, market, drive, ...
18941    [investigate, nyse, so, globe, today, announce...
3578     [salesforcecom, inc, form, file, reflect, inte...
3203     [wire, today, announce, former, member, member...
1464     [it, globe, today, announce, joined, company, ...
15633    [prnewswire, leader, ownership, development, m...
61       [unaudited, fiscal, year, prnewswire, signific...
8913     [wire, zayo, direct, subsidiary, global, provi...
11011    [help, plant, produce, low, carbon, support, g...
Name: Article, dtype: object

In [ ]:
X_test.head(10)

7637     [by, leader, big, pharma, aposs, cancer, race,...
16822                   [follow, press, release, in, with]
18162    [by, after, market, close, schedule, announce,...
10523    [announces, tax, treatment, distribution, prne...
5812     [azure, base, help, wire, in, grow, migration,...
118      [globe, nasdaq, apri, biopharmaceutical, compa...
8502     [by, say, online, plan, create, full, time, jo...
21030    [more, more, more, second, globe, nasdaq, ktcc...
982      [globe, nasdaq, xog, oil, gas, exploration, pr...
21355    [put, little, extra, pressure, say, aposs, mak...
Name: Article, dtype: object

#### Bag of Words (BoW) representation

In [ ]:
def build_bow_matrix(
    articles: pd.Series,
    vectorizer: Optional[CountVectorizer] = None,
    *,
    max_df: float = 0.95,
    min_df: int = 2,
    stop_words: str = "english"
) -> Tuple:
    """
    Convert a Series of token-lists into a Bag-of-Words matrix.

    Parameters
    ----------
    articles   : pd.Series
        Each element is a list of tokens (words) for one document.
    vectorizer : CountVectorizer or None
        • None  → fit a new CountVectorizer on `articles` and return it.
        • existing CountVectorizer → skip fitting and only transform.
    max_df, min_df, stop_words
        Passed straight to CountVectorizer (ignored if a pre-fitted
        `vectorizer` is supplied).

    Returns
    -------
    X          : scipy sparse matrix  (n_docs × |V|)
    vectorizer : the fitted CountVectorizer (use for future transforms)
    """
    # 1  join each token list into a single space-separated string
    article_text = articles.apply(" ".join)

    # 2  fit or transform
    if vectorizer is None:
        vectorizer = CountVectorizer(max_df=max_df, min_df=min_df, stop_words=stop_words)
        X = vectorizer.fit_transform(article_text)
    else:
        X = vectorizer.transform(article_text)

    return X, vectorizer

In [ ]:
# ---------------------------------------------
# X_train, X_test are the token-list Series
# produced earlier by the train_test_split step
# ---------------------------------------------

# Fit on training data
X_train_bow, bow_vect = build_bow_matrix(X_train)

# Transform held-out test data with the *same* vocabulary
X_test_bow,  _        = build_bow_matrix(X_test, vectorizer=bow_vect)

In [ ]:
print("Train Document-term matrix created. Shape:", X_train_bow.shape)
print("Test Document-term matrix created. Shape:", X_test_bow.shape)
print("Output – a SciPy sparse matrix shape = (n_docs, n_unique_train_set_terms) whose (i, j) entry is the raw count of term j in document i.")

Train Document-term matrix created. Shape: (8378, 18033)
Test Document-term matrix created. Shape: (2793, 18033)
Output – a SciPy sparse matrix shape = (n_docs, n_unique_train_set_terms) whose (i, j) entry is the raw count of term j in document i.


In [ ]:
# Convert sparse matrix to DataFrame
df_train_bow = pd.DataFrame(
	X_train_bow.toarray(),
	columns=bow_vect.get_feature_names_out()
)

df_test_bow = pd.DataFrame(
	X_test_bow.toarray(),
	columns=bow_vect.get_feature_names_out()
)

# Save to CSV
df_train_bow.to_csv('bow_train_matrix.csv')
print("Document-term matrix saved to bow_train_matrix.csv")

# Save to CSV
df_test_bow.to_csv('bow_test_matrix.csv')
print("Document-term matrix saved to bow_test_matrix.csv")



Document-term matrix saved to bow_train_matrix.csv
Document-term matrix saved to bow_test_matrix.csv


#### Simple Average & TF-IDF GloVe representation
A bag-of-words matrix captures frequency but not semantics—“car” and “automobile” are orthogonal. Word-vector models like GloVe collapse words into dense real-valued vectors where geometric proximity encodes meaning.

In [ ]:
# 0 Load pre-trained GloVe once (100-d here — change path if needed)
# ------------------------------------------------------------------
def load_glove(path: str) -> Dict[str, np.ndarray]:
    out = {}
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            parts = line.rstrip().split()
            out[parts[0]] = np.asarray(parts[1:], dtype=np.float32)
    return out

GLOVE_PATH   = r"C:\Users\gpapa\OneDrive\My Life\Education\2025 YALE MMS\Term_4S\T4_E_Adv_Lin_Algebra_p2\final_project\GloVe Embeddings\glove.6B.100d.txt"
glove        = load_glove(GLOVE_PATH)
EMBED_DIM    = 100
UNK          = np.zeros(EMBED_DIM, dtype=np.float32)

In [ ]:
# 1 Fit document-frequency stats **on training only**
# ------------------------------------------------------------------
def fit_vocab_and_idf(
    docs: Sequence[Sequence[str]],
    min_df: int   = 2,
    max_df_ratio: float = 0.95
) -> Tuple[set, Dict[str, float]]:
    """Return {kept tokens}, and IDF dict computed on `docs`."""
    N = len(docs)
    df_counts = Counter()
    for tokens in docs:
        df_counts.update(set(tokens))

    keep = {tok for tok, df in df_counts.items()
            if min_df <= df <= max_df_ratio * N}

    idf  = {tok: np.log(1 + N / (1 + df_counts[tok])) for tok in keep}
    return keep, idf

In [ ]:
# 2 Embed a list-of-tokens article  (avg  or  tf-idf weighted)
# ------------------------------------------------------------------
def embed_doc(tokens: Sequence[str],
              *,
              kind: str,                 # "avg"  or  "tfidf"
              keep: set,
              idf: Dict[str, float] = None
) -> np.ndarray:
    toks = [t for t in tokens if t in keep]
    if not toks:
        return UNK

    if kind == "avg":
        return np.mean([glove.get(t, UNK) for t in toks], axis=0)

    if kind == "tfidf":
        tf = Counter(toks)
        total_doc_tokens = len(toks)
        weights, scaled_word_vecs = [], []
        for t in toks:
            weight = (tf[t] / total_doc_tokens) * idf.get(t, 0.0)
            weights.append(weight)
            scaled_word_vecs.append(glove.get(t, UNK) * weight)
        w_sum = sum(weights)
        return UNK if w_sum == 0 else np.sum(scaled_word_vecs, axis=0) / w_sum

    raise ValueError("kind must be 'avg' or 'tfidf'")


In [ ]:
# 3 Pipeline – from tokens to four matrices
# ------------------------------------------------------------------
def build_glove_matrices(
    X_train: pd.Series,    # Series of token lists
    X_test:  pd.Series
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:

    keep, idf = fit_vocab_and_idf(X_train)

    # helper lambdas avoid repeated kwargs
    avg   = lambda toks:   embed_doc(toks, kind="avg",   keep=keep)
    tfidf = lambda toks:   embed_doc(toks, kind="tfidf", keep=keep, idf=idf)

    Xtr_avg  = np.vstack(X_train.apply(avg).values)
    Xte_avg  = np.vstack(X_test.apply(avg).values)

    Xtr_tf   = np.vstack(X_train.apply(tfidf).values)
    Xte_tf   = np.vstack(X_test.apply(tfidf).values)

    return Xtr_avg, Xte_avg, Xtr_tf, Xte_tf

In [ ]:
# 4 Run it
# ------------------------------------------------------------------
X_train_glove_avg,  X_test_glove_avg, \
X_train_glove_tfidf, X_test_glove_tfidf = build_glove_matrices(X_train, X_test)

print("Shapes  –  Train avg:",   X_train_glove_avg.shape, "Test avg:",  X_test_glove_avg.shape)
print("Output – a NumPy array shape = (n_docs, n_embedding_dim) whose (i,j) entry is the simple avg embedding of characteristic (j) in document (i).")
print('\n')
print("Shapes  – Train tfidf:", X_train_glove_tfidf.shape, "Test tfidf:", X_test_glove_tfidf.shape)
print("Output – a NumPy array shape = (n_docs, n_embedding_dim) whose (i,j) entry is the tfidf embedding of characteristic (j) in document (i)")

Shapes  –  Train avg: (8378, 100) Test avg: (2793, 100)
Output – a NumPy array shape = (n_docs, n_embedding_dim) whose (i,j) entry is the simple avg embedding of characteristic (j) in document (i).


Shapes  – Train tfidf: (8378, 100) Test tfidf: (2793, 100)
Output – a NumPy array shape = (n_docs, n_embedding_dim) whose (i,j) entry is the tfidf embedding of characteristic (j) in document (i)


**Shapes  –  Duplicates Train avg: (16203, 100) Duplicates Test avg: (5401, 100)  
Output – a NumPy array shape = (n_docs, n_embedding_dim) whose (i,j) entry is the simple avg embedding of characteristic (j) in document (i).**   


**Shapes  – Duplicates Train tfidf: (16203, 100) Duplicates Test tfidf: (5401, 100)  
Output – a NumPy array shape = (n_docs, n_embedding_dim) whose (i,j) entry is the tfidf embedding of characteristic (j) in document (i)**

In [ ]:
# 5 Display
# -----------------------------------------------------------------
print("First 5 rows and 10 columns of the train avg matrix:")
print(X_train_glove_avg[:5, :5])
print('\n')
print("First 5 rows and 10 columns of the train tfidf matrix:")
print(X_train_glove_tfidf[:5, :5])

First 5 rows and 10 columns of the train avg matrix:
[[ 0.07170191  0.06997532  0.16436417 -0.02037911  0.08340277]
 [-0.01395935  0.05536129  0.26111835  0.01325093  0.00734193]
 [ 0.03635607  0.02148002  0.19563235 -0.02306763  0.01742517]
 [ 0.07147374  0.12722564  0.18284309 -0.01860794  0.11183619]
 [-0.04374495  0.05594995  0.12998235  0.01704939  0.05162265]]


First 5 rows and 10 columns of the train tfidf matrix:
[[ 0.16753436 -0.04237869  0.24857812 -0.01005682  0.14047462]
 [ 0.07926302  0.04166919  0.35364992 -0.09686092 -0.06041234]
 [ 0.03443183 -0.0550491   0.20656885 -0.02302127  0.00702674]
 [ 0.06877818  0.08092811  0.14684223 -0.1043917   0.18785022]
 [ 0.00657955  0.04952998 -0.0248589   0.02433235  0.07496377]]


First 5 rows and 5 columns of the train avg matrix:  
[[ 0.06743193  0.08372917  0.19575463 -0.04276435  0.01124586]  
 [ 0.08235665  0.08270617  0.19840498 -0.02022048  0.06879219]  
 [-0.2032155   0.22544767  0.14584434  0.09497339  0.101927  ]  
 [ 0.14726     0.088411    0.28386    -0.2797     -0.32498   ]  
 [ 0.03238507  0.06798428  0.22245008 -0.03258403  0.1515859 ]]  
  

First 5 rows and 10 columns of the train tfidf matrix:  
[[ 0.1029514  -0.00972255  0.1807258  -0.02345218 -0.01122584]  
 [ 0.15955492  0.01012329  0.21545029 -0.00057363  0.11649812]  
 [-0.22639619  0.24005397  0.10130514  0.04625234  0.02976975]  
 [ 0.14726     0.088411    0.28386    -0.27970001 -0.32497999]  
 [ 0.11210106  0.00506607  0.23441343 -0.04273045  0.1871462 ]]  

In [ ]:
# 6 Build tidy DataFrames from the four matrices returned earlier
# ---------------------------------------------------------------

df_train_glove_avg   = pd.DataFrame(X_train_glove_avg,
                                    columns=[f"dim_{i}" for i in range(EMBED_DIM)])
df_test_glove_avg    = pd.DataFrame(X_test_glove_avg,
                                    columns=[f"dim_{i}" for i in range(EMBED_DIM)])

df_train_glove_tfidf = pd.DataFrame(X_train_glove_tfidf,
                                    columns=[f"dim_{i}" for i in range(EMBED_DIM)])
df_test_glove_tfidf  = pd.DataFrame(X_test_glove_tfidf,
                                    columns=[f"dim_{i}" for i in range(EMBED_DIM)])


# 7 Optional: save to disk (one file per split/weighting scheme)
# ---------------------------------------------------------------
df_train_glove_avg.to_csv("glove_train_avg.csv",   index=False)
df_test_glove_avg.to_csv("glove_test_avg.csv",    index=False)

df_train_glove_tfidf.to_csv("glove_train_tfidf.csv", index=False)
df_test_glove_tfidf.to_csv("glove_test_tfidf.csv",  index=False)

print("Saved: glove_train_avg.csv, glove_test_avg.csv, "
      "glove_train_tfidf.csv, glove_test_tfidf.csv")

Saved: glove_train_avg.csv, glove_test_avg.csv, glove_train_tfidf.csv, glove_test_tfidf.csv


##### Old
- Used the full data['Article'] everywhere  
- Hard-wired winsorisation inside main loop  
- Separate code blocks for avg vs tf-idf  
- Generated just one matrix  
- Globals sprinkled in notebook
  
##### New  
- Fits vocab & IDF only on X_train → avoids test leakage  
- Extracted to fit_vocab_and_idf (re-usable & test-safe)  
- Unified via embed_doc(kind="avg"/"tfidf")  
- Now returns four: train/test × (avg, tf-idf)  
- Functions make the pipeline callable from anywhere  